In [1]:
%cd ..
%load_ext nb_black

/home/hardianlawi/attention-lstm-sentiment-analysis


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import tensorflow as tf
from src.preprocess import Preprocessor
from src.data import get_data, ID2WORD, OOV_ID
from src.models import get_model


vocabulary_size = 5000
embedding_size = 32
max_words = 500

(X_train, y_train), (X_test, y_test) = get_data(vocabulary_size)

print("Maximum review length: {}".format(len(max((X_train + X_test), key=len))))
print("Minimum review length: {}".format(len(min((X_train + X_test), key=len))))


preprocessor = Preprocessor(maxlen=max_words, oov_token=ID2WORD[OOV_ID])
preprocessor.fit_on_texts(X_train + X_test)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)


Maximum review length: 2494
Minimum review length: 7


<IPython.core.display.Javascript object>

In [3]:
model, model_attention = get_model(
    "attention", max_words, vocabulary_size, embedding_size
)
model.summary(line_length=200)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
input_1 (InputLayer)                                                                      [(None, 500)]                                                                   0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
embedding (Embedding)                                                                     (None, 500, 32)                                                                 160000     

<IPython.core.display.Javascript object>

In [4]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

model.fit(
    X_train2,
    y_train2,
    validation_data=(X_valid, y_valid),
    batch_size=batch_size,
    epochs=num_epochs,
)

Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 235s 9ms/sample - loss: 0.4249 - accuracy: 0.7949 - val_loss: 0.2066 - val_accuracy: 0.9219
Epoch 2/3
24936/24936 [==============================] - 259s 10ms/sample - loss: 0.2401 - accuracy: 0.9058 - val_loss: 0.2073 - val_accuracy: 0.8906
Epoch 3/3
24936/24936 [==============================] - 234s 9ms/sample - loss: 0.2059 - accuracy: 0.9220 - val_loss: 0.1949 - val_accuracy: 0.9219


<IPython.core.display.Javascript object>

In [5]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap="viridis")
    fontdict = {"fontsize": 14}
    ax.set_xticklabels([""] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([""] + predicted_sentence, fontdict=fontdict)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    plt.show()

<IPython.core.display.Javascript object>

In [6]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", scores[1])

Test accuracy: 0.88328


<IPython.core.display.Javascript object>

In [15]:
import json
import pickle
from typing import List

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json


class Preprocessor(object):
    def __init__(self, maxlen, oov_token):
        self._tokenizer = Tokenizer(oov_token=oov_token)
        self._maxlen = maxlen

    def fit_on_texts(self, texts):
        self._tokenizer.fit_on_texts(texts)

    def transform(self, sentences: List[str]):
        sequences = self._tokenizer.texts_to_sequences(sentences)
        padded_sequences = sequence.pad_sequences(sequences, maxlen=self._maxlen)
        return padded_sequences

    def save(self, path):
        with open(path, "wb") as handle:
            pickle.dump(self, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
    @classmethod
    def load(cls, path):
        with open(path, 'rb') as handle:
            return pickle.load(handle)


(X_train, y_train), (X_test, y_test) = get_data(vocabulary_size)

print("Maximum review length: {}".format(len(max((X_train + X_test), key=len))))
print("Minimum review length: {}".format(len(min((X_train + X_test), key=len))))


preprocessor = Preprocessor(maxlen=max_words, oov_token=ID2WORD[OOV_ID])
preprocessor.fit_on_texts(X_train + X_test)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)


Maximum review length: 2494
Minimum review length: 7


<IPython.core.display.Javascript object>

In [16]:
preprocessor.save("preprocessor.pkl")

<IPython.core.display.Javascript object>

In [17]:
temp = Preprocessor.load("preprocessor.pkl")

<IPython.core.display.Javascript object>

In [18]:
temp

<IPython.core.display.Javascript object>